In [1]:
import boto3
import pandas as pd
import numpy as np
import re

In [2]:
client = boto3.client('s3')

# sequence file

path = 's3://transcriptional-effect-adversity-in-hippocampus/sra_metadata_07dae975-66fa-4503-a5ad-bdbc99bb2fef.csv'

path_sample = 's3://transcriptional-effect-adversity-in-hippocampus/sample_1.fastq'

path_13062383 = 's3://transcriptional-effect-adversity-in-hippocampus/SRR13062383_1.fastq'

In [3]:
df = pd.read_csv(path_sample, header=None, sep='\n')
# df = pd.read_csv(path_sample, error_bad_lines=False) # This skips the line with error when loading data
pd.set_option('display.max_colwidth', None)
df.head(10)

,0
0,@SRR13062383.1 1 length=150
1,CCCTGCCATCCGGAACCACGACAAGGTCCTGCAGCGCCTGTGTGAGCACTTGGACCACGCCCTGTTGTACGGATTGCAAGACCTGTCCTCTGGCTACTGGGTGGTCGTGGTCCATTTCACCCGCAGAGAGGCCATCAGGCAGATCGGAAG
2,+SRR13062383.1 1 length=150
3,BBBDFFFFDHHFDCGHGIJGIIJJFDFHHIIJJHHGIJHIGIHIJJJIJGGHHHHFEFDDDDBDDDDCCDDBDDDDDDDCDDDDDDDDDDDDDDDDDDCDD95>5CD@DB?CCCEEEEDDDDDDDDDDCDBDDDDDDCCDDDDDDDDDDB
4,@SRR13062383.2 2 length=150
5,NTCGACTCCATTAAAGACTTTTCTTTAAAATAGCAGAGCCTGAACCTTGATATCCAAGCCTTAGCCTGCCCGTCCGCCCTGCCTATGATCATAGAGCTCCATCGGTTCAAAGAACCCCAGACATCGGCAGCCCACCCCTCTGAACTCCAG
6,+SRR13062383.2 2 length=150
7,"#1=+<+@)2@2,,+<+2<<C>7,33@+?7+2+*:*@:?3?7))**:3A*0*0?BB###############################################################################################"
8,@SRR13062383.3 3 length=150
9,NTCCTTCTCTCTCTGGTTTAACTTATCTTGTACTAGAAAAGCCAGGGAGGTGGTAAATGTATGGTGGTGTTTTGCTTACAATAGTTCACAAAAATCTCTCTCTAGGATGAGATGCTGGCACTGACGAACATTGCCTAAGATTTCTATGAA


In [4]:
# bed file

path_local_bed1 = "./Chip-seq/GSM2228982_42YVE_s_5_sequence_norm.bed/GSM2228982_42YVE_s_5_sequence_norm.bed"

df_bed_chr1 = pd.read_csv(path_local_bed1, header=None, sep='\n')
# df = pd.read_csv(path_sample, error_bad_lines=False) # This skips the line with error when loading data
pd.set_option('display.max_colwidth', None)
df_bed_chr1.head(10)

,0
0,chr1\t67926\t67962\t0\t1\t+
1,chr1\t97370\t97406\t0\t1\t+
2,chr1\t101069\t101105\t0\t1\t-
3,chr1\t132314\t132350\t0\t1\t-
4,chr1\t156509\t156545\t0\t1\t-
5,chr1\t157123\t157159\t0\t1\t+
6,chr1\t165094\t165130\t0\t1\t+
7,chr1\t197984\t198020\t0\t1\t+
8,chr1\t223687\t223723\t0\t1\t-
9,chr1\t234721\t234757\t0\t1\t+


In [5]:
# sequence file

path_fna = "./Rattus norvegicus genome/GCA_015227675.2_mRatBN7.2_genomic.fna"
sequence = ""
seq = ""
with open(path_fna) as f:
    lines = f.readlines()
    # cut off the first line of the sequence file
    lines = lines[1:]
for line in lines:
    sequence += line[:-1]
f.close()
# print(sequence)

# seq with re
#     sequence = f.read()
# # print(sequence)
# match = re.search(r"(\w+)(\n)(\w+)", sequence)
# if match:
#     seq = match.groups(1)
# print(seq)

# seq as a df
# df_seq = pd.read_csv(path_fna, header=None, sep='\n')
# pd.set_option('display.max_colwidth', None)
# df_seq.head(10)


In [6]:
'''
UTILITY FUNCTIONS
'''

# 1. TAKE COMPLEMENT OF A SEQUENCE
def take_complement(seq=""):
    new_seq = ""
    for char in seq:
        if char == 'A' or char == 'a':
            new_seq += 'T'
        elif char == 'T' or char == 't':
            new_seq += 'A'
        elif char == 'G' or char == 'g':
            new_seq += 'C'
        elif char == 'C' or char == 'c':
            new_seq += 'G'
    return new_seq

# 2. REVERSE STRING
def reverse_str(str):
    # print("str=",str," reversed=",str[::-1])
    return str[::-1]
# def reverse_str(str):
#     new_str = ""
#     size = len(str)
#     for idx in range(size):
#         new_str += str[size-idx-1:size-idx]
#     # print("str=",str," reversed=",new_str)
#     return new_str

PARSING DATA

In [9]:
'''
PARSE BED FILES
'''

df_bed = df_bed_chr1
seq_arr_chr1 = list()
seq_arr = seq_arr_chr1

labels = list()
entries = list()                  # list( dict( str: list(str, str, str) ) )

# Example: line = ["chr1", "67926", "67962", "0", "1", "+"]
for line in df_bed.to_numpy():
    # if line and len(line) > 1:
    line = line.tolist()[0].split("\t")
    # print("line=", line)
    if line[0] not in labels:
        labels.append(line[0])
    d = dict()
    #------------------------------------------------------------
    # dict = {chr#, [begin, end, +/-]}
    #------------------------------------------------------------
    d[line[0]] = {line[1], line[2], line[5]}
    entries.append(d)

    if line[5] == "+":
        seq_arr.append(sequence[int(line[1]) : int(line[2])+1])
        # print(seq_arr)
    elif line[5] == "-":
        sub_seq = sequence[int(line[1]) : int(line[2])+1]
        # sub_seq.reverse()
        reversed = reverse_str(sub_seq)
        reversed_complement = take_complement(reversed)
        seq_arr.append(reversed_complement)
    else:
        seq_arr.append("-")

print(seq_arr[:20])


['gggggaggcgttagggggatgttggcctggcaAACTG', 'catccaatggaaaaaagacagcattttcagcaaatgg', 'TCCCTTCCATTTATGATTTTGTTAATTATGATAATTA', 'TAAGGCCTCTATCTGTTGTAGGATTGGTAAAGATCTA', 'TTGGTGATCACTGTTTGACTCCATGCCATCTCCCTCA', 'ACAGTCAAGAGCTcaacaaatctattttctttaaaaa', 'TTTCATGCCTGCAAAGATAGATAGTTGGCCCATGgga', 'atttatctcttctcgatctagtaaggttttcatcttc', 'GGAGGGGTTGGGTTGATGTTGGCATGAAAACTGGGAA', 'acaagaattcccttggcaggtaatcgagaggcaaaga', 'CTGCAACCAGGAGGACCTGCACCGCCTTTTCCAGGAG', 'CCAGTCCTATCCAAGTCCATTCAAAATGAAACATCAA', 'gtttagtctctgggagcactgggtgttgacatttttg', 'AGCCTCCCTCTCCCCTGATCTCAGCATTCACTGTGAA', 'aacaacttcggcaaagttgatggatataaaatcaact', 'CTCCCACCAAAGCAAACACTGAATATCCAAACACACC', 'catatttttaaaattaagaaaaatattgaatatctAC', 'AAAGTCAGAAATAACTCCCTAAGGAAATATGGGACAA', 'AGTGATGTTATTTACTGGCTTGCTTACTTCCCCTGGC', 'ggaggaggagactaggacagaaaggaagaggcaggag']


In [8]:
# '''
# PARSE FNA FILE
# CONVERT THE DATA FRAME TO A NUMPY ARRAY:

# array([['@SRR13062383.1 1 length=150'], ['CCCTGCCATCCGGAACCACGACAAGGTCCTGCAGCGCCTGTGTGAGCACTTGGACCACGCCCTGTTGTACGGATTGCAAGACCTGTCCTCTGGCTACTGGGTGGTCGTGGTCCATTTCACCCGCAGAGAGGCCATCAGGCAGATCGGAAG'],

# ['+SRR13062383.1 1 length=150'],['BBBDFFFFDHHFDCGHGIJGIIJJFDFHHIIJJHHGIJHIGIHIJJJIJGGHHHHFEFDDDDBDDDDCCDDBDDDDDDDCDDDDDDDDDDDDDDDDDDCDD95>5CD@DB?CCCEEEEDDDDDDDDDDCDBDDDDDDCCDDDDDDDDDDB'],

# ...

# ])

# '''

# SEPARATE HEADER LINES AND SEQUENCE LINES

# headers = list()
# sequence = list()

# for i in range(df.to_numpy().shape[0]):
#     # print(type(df.to_numpy()[i].tolist()[0]))
#     if i % 2 == 0:
#         headers.append(df.to_numpy()[i].tolist()[0])
#     else:
#         sequence.append(str(df.to_numpy()[i].tolist()[0]))
# headers = np.asarray(headers)
# sequence = np.asarray(sequence)

# headers
# sequence





# # CHECK SEQUENCE VALIDITY (DELETE NON-ALPHABETIC SEQUENCES)

# valid_chars = ["A", "T", "C", "G"]

# # eliminate non-alphabets
# # for idx in range(len(sequence)):
# idx = 0
# while idx <= len(sequence):
#     if idx >= len(sequence):
#         break
#     for char in sequence[idx]:
#         match = re.match('[A-z]', char)
#         # if char not in alphabets:
#         if not match:
#             headers = np.delete(headers, idx)
#             sequence = np.delete(sequence, idx)
#             idx -= 1
#             break
#     idx += 1

# sequence